In [3]:
import tensorflow as tf
from tensorflow.keras import datasets
import numpy as np
import cv2
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Preprocessing
def preprocess(images):
    images = images.astype('float32') / 255.0
    return images

train_images = preprocess(train_images)
test_images = preprocess(test_images)

# Feature extraction using SIFT
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []
    for image in images:
        image = (image * 255).astype(np.uint8)  # Convert image to uint8
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None and len(keypoints) > 0:  # Ensure valid descriptors
            descriptors_list.append(descriptors)
            keypoints_list.append(keypoints)
    return keypoints_list, descriptors_list

train_keypoints, train_descriptors = extract_sift_features(train_images)
test_keypoints, test_descriptors = extract_sift_features(test_images)

# Flatten descriptors
train_descriptors_flat = np.vstack(train_descriptors)
test_descriptors_flat = np.vstack(test_descriptors)

# K-means clustering
kmeans = KMeans(n_clusters=100)
kmeans.fit(train_descriptors_flat)

# Generate Bag-of-Visual Words representation
def bow_representation(keypoints, descriptors, kmeans):
    bow_representation = np.zeros((len(keypoints), kmeans.n_clusters), dtype=np.float32)
    for i in range(len(keypoints)):
        if descriptors[i] is not None:
            words = kmeans.predict(descriptors[i])
            for word in words:
                bow_representation[i, word] += 1
    return bow_representation

train_bow = bow_representation(train_keypoints, train_descriptors, kmeans)
test_bow = bow_representation(test_keypoints, test_descriptors, kmeans)

# Scale features
scaler = StandardScaler()
train_bow_scaled = scaler.fit_transform(train_bow)
test_bow_scaled = scaler.transform(test_bow)

# SVM classifier
svm = SVC(kernel='linear')
svm.fit(train_bow_scaled, train_labels[:len(train_bow)].ravel())

# Predictions
train_predictions = svm.predict(train_bow_scaled)
test_predictions = svm.predict(test_bow_scaled)

# Accuracy
train_accuracy = accuracy_score(train_labels[:len(train_bow)], train_predictions)
test_accuracy = accuracy_score(test_labels[:len(test_bow)], test_predictions)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


Training Accuracy: 0.12475953831356204
Test Accuracy: 0.10485164394546913
